In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
food_name = []
with open('../raw_data/UECFOOD100/category.txt', 'r') as f:
    next(f)
    for line in f:
        line = line.strip().split('\t')
        food_name.append(line[1])

In [3]:
len(food_name)

100

In [4]:
import pandas as pd
import requests
import numpy as np

def get_nutrition(food_name):
    nutrition_data = pd.DataFrame(columns=['name', 'protein', 'calcium', 'fat', 'carbohydrates', 'vitamins'])
    carbs = 0
    vitamin_a = 0
    vitamin_c = 0
    data_to_concat = []
    for name in food_name:
        try:
            url = "https://api.nal.usda.gov/fdc/v1/foods/search?api_key=d4D6dSOc81pTAOY2gsNZ0YhjkMlhStLJRoII5SJu&query=" + name
            response = requests.get(url)
            data = response.json()
            flatten_json = pd.json_normalize(data["foods"])
            if not flatten_json.empty:
                first_food = flatten_json.iloc[0]
                first_food_nutrition_list = first_food.foodNutrients

                data_to_concat = []  # moved inside try block
                for item in first_food_nutrition_list:
                    if item['nutrientNumber'] == "203":
                        protein = item['value']
                        continue
                    if item['nutrientNumber'] == "301":
                        calcium = item['value']
                        continue
                    if item['nutrientNumber'] == "204":
                        fat = item['value']
                        continue
                    if item['nutrientNumber'] == "205":
                        carbs = item['value']
                        continue
                    if item['nutrientNumber'] == "318":
                        vitamin_a = item['value']
                        continue
                    if item['nutrientNumber'] == "401":
                        vitamin_c = item['value']
                        continue

                vitamins = float(vitamin_a) + float(vitamin_c)
                data_to_concat.append({
                    'name': name,
                    'protein': protein or np.nan,
                    'calcium': calcium / 1000 if calcium else np.nan,
                    'fat': fat or np.nan,
                    'carbohydrates': carbs or np.nan,
                    'vitamins': vitamins / 1000 if vitamins else np.nan
                })
            
                nutrition_data = pd.concat([nutrition_data, pd.DataFrame(data_to_concat)], ignore_index=True)
            else:
                print(f"Sorry {name}, is not in the database. But it's propably healthy. Enjoy! :)")
        except KeyError:
            print(f"Sorry {name}, is not in the database. But it's propably healthy. Enjoy! :)")

    return nutrition_data


In [5]:
nutrition101 = get_nutrition(food_name)

Sorry takoyaki, is not in the database. But it's propably healthy. Enjoy! :)
Sorry ganmodoki, is not in the database. But it's propably healthy. Enjoy! :)
Sorry nanbanzuke, is not in the database. But it's propably healthy. Enjoy! :)


In [6]:
nutrition101.shape

(97, 6)

In [ ]:
nutrition101.loc[nutrition101['name'] == 'Japanese-style pancake']